<a href="https://colab.research.google.com/github/enjoyPG/2023Gifted/blob/main/students/DolBaKi/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/5-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###앙상블 학습
  * 정형 데이터를 다루기에 최적화되어있는 학습
  * 더 좋은 예측 결과를 내기 위해 여러 모델 훈련하는 머신러닝 알고리즘
  * 대부분 결정트리 기반
  * 데이터량이 적당할 때 많이 사용

###부트스트랩
  * 데이터 세트 중복 허용 데이터 샘플링 방식
  * 특성 수^(1/2)만큼 랜덤 선택
  * 남는 샘플을 OOB라고 함
  * OOB점수 사용시 교차검증 대신 가능, 훈련세트 효율 증가


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine=pd.read_csv('https://bit.ly/wine_csv_data')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()
train_input,test_input,train_target,test_target=train_test_split(data, target,test_size=0.2,random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(rf, train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


전보다 더 줄음

과대적합률 감소, 조금 더 많은 특성들이 훈련에 기여 가능

In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


###엑슽으라틀이
 * 랜덤포레스트와 차이:**부트스트랩 사용안함**
 * 엑스트라 트리가 사용하는 결정 트리:splitter='random'
 * 랜덤하게 노드 분할



In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1) # cross_validate는 교차 검증
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


###Gradient Boost
 * 깊이 얕은 결정트리 사용해 이전 트리의 오차를 보완하는 방식
 * 깊이가 얕아 **과대적합에 강하고 높은 일반화 성능**을 가짐

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


#진짜 강하다

In [ ]:
gb=GradientBoostingClassifier(n_estimators=500,learning_rate=0.2,random_state=42)
scores=cross_validate(gb,train_input,train_target,return_train_score=True,n_jobs=-1)

print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input,train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


###힛읕오깁안 붓읕잉
 * 미리 256개 구간으로 나눠서 노드 분할 시 최적분할을 매우 빠르게 찾을 수 있음
 * 입력 누락되어도 따로 전처리할 이유 없음
 * 빠르고 편리해서 인기 많음

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [ ]:
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result=permutation_importance(hgb,train_input,train_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [ ]:
result=permutation_importance(hgb,test_input,test_target,n_repeats=10,random_state=42,n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [ ]:
hgb.score(test_input,test_target)

0.8723076923076923

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier(tree_method='hist',random_state=42)
scores=cross_validate(xgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


In [ ]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier(random_state=42)
scores = cross_validate(lgb,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


### 사이킷런
 * 랜덤 포레스트 : 부트스트랩 샘플 사용. 대표 앙상블 학습 알고리즘
 * 엑스트라 트리 : 결정 트리의 노드를 랜덤하게 분할
 * 그레이디언트 부스팅 : 이전 트리 손실 보완하는 식으로 얕은 결정트리 연속 추가
 * 히스토 기반 그레이디언트 부스팅 : 훈련 데이터 256개 정수 구간으로 나눠 빠르고 높은 성능냄